In [19]:
import vatect as vt
import os
import pandas as pd
import glob

In [20]:
meta_file = "/Users/lye1/data/projects_akdemirlab/catalyst_main/meta/summary_2025_01_17.csv"
meta = pd.read_csv(meta_file)

directory_path = "/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/*/*.copynumber.totalCn.bw"
files = [file for file in glob.glob(directory_path)]

paths0 = [os.path.join(directory_path, file) for file in files]
names0 = [os.path.basename(file).replace(".copynumber.totalCn.bw", "") for file in files]
meta_ploidy = pd.merge(meta[['wgs_seq_id', 'ploidy_ascat']], pd.DataFrame({'wgs_seq_id':names0, 'paths':paths0}), on='wgs_seq_id', how='inner')
#replace adjusted sample 235860-WG01
meta_ploidy.loc[meta_ploidy['wgs_seq_id']=='235860-WG01','paths']='/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/ascat_adj/235860-WG01/235860-WG01.copynumber.totalCn.bw'
meta_ploidy.loc[meta_ploidy['wgs_seq_id']=='235860-WG01','ploidy_ascat']=2.99737857818088

paths = list(meta_ploidy['paths'])
names = list(meta_ploidy['wgs_seq_id'])
ploidy = list(meta_ploidy['ploidy_ascat'])

In [21]:
print(paths)
print(names)
print(ploidy)
print((meta_ploidy[meta_ploidy['wgs_seq_id']=='235860-WG01']))

['/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/236626-WG01/236626-WG01.copynumber.totalCn.bw', '/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/235842-WG01/235842-WG01.copynumber.totalCn.bw', '/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/235843-WG01/235843-WG01.copynumber.totalCn.bw', '/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/237614-WG01/237614-WG01.copynumber.totalCn.bw', '/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/237615-WG01/237615-WG01.copynumber.totalCn.bw', '/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/236627-WG01/236627-WG01.copynumber.totalCn.bw', '/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/237616-WG01/237616-WG01.copynumber.totalCn.bw', '/Volumes/scratch/neurosurgery/akdemirlab/catalyst_main/results/summary_v2/ascat/235844-WG01/235

In [22]:
meta_ploidy[meta_ploidy['wgs_seq_id']=='235860-WG01']

,wgs_seq_id,ploidy_ascat,paths
27,235860-WG01,2.997379,/Volumes/scratch/neurosurgery/akdemirlab/catal...


In [23]:

gap_file = "/Users/lye1/data/projects_akdemirlab/catalyst_main/reference/hgTables_gap_hg38.tsv"
gap = pd.read_csv(gap_file, sep = '\t')
#genes = pd.read_csv("driver_gene_cnv_location.tsv", sep='\t')
genes = pd.read_csv('/Users/lye1/data/projects_akdemirlab/catalyst_main/data/download/ensembl_grch38p14_gene_location.csv')

genes = genes[genes['chr'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
'11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
'21', '22', 'X', 'Y'])]
genes = genes[genes['name'].notnull()]
genes['chr'] = 'chr'+ genes['chr'] 

In [ ]:
matrix_arms= vt.cn_matrix_arms(paths,  names, ploidy)
matrix_bins = vt.cn_matrix_bins(paths,  names, ploidy, bins_size=10_000_000)
matrix_bins_filtered = vt.filter_gaps(matrix_bins,gap)
matrix_genes = vt.cn_matrix_genes(paths,  names, ploidy, genes)

In [9]:
matrix_bins_filtered.to_csv('/Users/lye1/data/projects_akdemirlab/catalyst_main/notebooks_s2/tables/cn_matrix_bin_10m.csv', index=False)
matrix_arms.to_csv('/Users/lye1/data/projects_akdemirlab/catalyst_main/notebooks_s2/tables/cn_matrix_arm.csv', index=False)

In [15]:
import numpy as np
def matrix2comut_all(matrix):
    df = pd.melt(matrix, id_vars=['chr', 'name', 'start', 'end'],
                                    var_name='sample', value_name='cn')
    #print(df[df['sample']=='235855-WG01'])
    df['cna'] = np.where(df['cn'] <= 1.5, 'deletion', np.where(df['cn'] >=2.5, 'amplification', ''))
    df = df[['name', 'sample', 'cna']].rename(columns={'name': 'category', 'cna': 'value'})
    #print(df[df['sample']=='235855-WG01'])
    return df

In [16]:
matrix2comut_all(matrix_arms).to_csv('/Users/lye1/data/projects_akdemirlab/catalyst_main/results/figure_comut_v2/comut_arm_wgs.csv', index=False)
matrix2comut_all(matrix_genes).to_csv('/Users/lye1/data/projects_akdemirlab/catalyst_main/results/figure_comut_v2/comut_genes_wgs.csv', index=False)

In [17]:
comut_all_arm_merged = matrix2comut_all(matrix_arms)
comut_all_arm_merged['category_merge'] = ''
comut_all_arm_merged.loc[(comut_all_arm_merged['category']=='chr1_p')|(comut_all_arm_merged['category']=='chr19_q'), 'category_merge'] = 'chr1p/19q'
comut_all_arm_merged.loc[(comut_all_arm_merged['category']=='chr7_p')|(comut_all_arm_merged['category']=='chr7_q'), 'category_merge'] = 'chr7'
comut_all_arm_merged.loc[(comut_all_arm_merged['category']=='chr10_p')|(comut_all_arm_merged['category']=='chr10_q'), 'category_merge'] = 'chr10'
comut_all_arm_merged = comut_all_arm_merged[comut_all_arm_merged['category_merge']!='']
comut_all_arm_merged['category'] = comut_all_arm_merged['category_merge']
comut_all_arm_merged = comut_all_arm_merged.drop(columns='category_merge')  
print(comut_all_arm_merged.head()) 

print('\n remove not-uniques:')
def check_consistency(group):
  if group.nunique() > 1:
    return ''
  else:
    return group.iloc[0]  # Return the first value (consistent)

# 'and'
## comut_all_arm_merged['value'] = comut_all_arm_merged.groupby(['category', 'sample'])['value'].transform(check_consistency)

comut_all_arm_merged = comut_all_arm_merged.drop_duplicates() 
print(comut_all_arm_merged)

comut_all_arm_merged.to_csv('/Users/lye1/data/projects_akdemirlab/catalyst_main/results/figure_comut_v2/comut_chr_wgs.csv', index=False)                                           

     category       sample          value
0   chr1p/19q  236626-WG01               
12       chr7  236626-WG01  amplification
13       chr7  236626-WG01               
18      chr10  236626-WG01       deletion
19      chr10  236626-WG01       deletion

 remove not-uniques:
       category       sample          value
0     chr1p/19q  236626-WG01               
12         chr7  236626-WG01  amplification
13         chr7  236626-WG01               
18        chr10  236626-WG01       deletion
37    chr1p/19q  236626-WG01  amplification
...         ...          ...            ...
4398  chr1p/19q  238066-WG01  amplification
4410  chr1p/19q  235920-WG01               
4422       chr7  235920-WG01  amplification
4428      chr10  235920-WG01       deletion
4447  chr1p/19q  235920-WG01       deletion

[327 rows x 3 columns]
